In [9]:
%%capture
!pip install xarray[complete] numpy dask-expr

In [1]:

import pandas as pd
import xarray as xr
import os
from dask.diagnostics import ProgressBar
from datetime import datetime
from dask import dataframe as dd

In [2]:
originalFiles = '/home/ec2-user/SageMaker/datalake/notebooks/utils/data/temporal/'
outputPath = '/home/ec2-user/SageMaker/datalake/notebooks/utils/data/brozen/'
variableName = 'Solar-Radiation-Flux'

def netcdfChange(path
                 , file
                 ,outputPath
                 ,variableName: str
                ):
    dt = file.split('_')[-2]
    dt = datetime.strptime(dt,'%Y%m%d')
    f = xr.open_dataset(path+file)
    f = f.drop_vars(['crs'])
    f = f.rename_vars(name_dict = {'Band1':variableName})
    #time_da = xr.DataArray({'time': dt})
    f = f.assign_coords(time = dt)
    f = f.expand_dims(dim="time")
    f.to_netcdf(outputPath+file)
    
    
    

In [3]:
getFile = os.listdir(originalFiles)
filespd= pd.DataFrame(getFile,columns=['file'])
filesDD = dd.from_pandas(filespd,npartitions=90)


In [4]:
with ProgressBar():
    filesDD.map_partitions(lambda df:
                          df.file.apply(lambda x: netcdfChange(path = originalFiles
                                                               ,file = x
                                                               ,outputPath=outputPath
                                                               ,variableName = variableName
                                                              )
                                       ),meta= {'name':'object'}
                          ).compute(scheduler='processes')

[########################################] | 100% Completed | 121.32 s
